# Test biologically plausible online non-negative ICA (BioNICA) network
Two algorithms implementing non-negative ICA from Lipshutz, Pehlevan, Chklovskii, 2022. 

Their implementation is available on Github: https://github.com/flatironinstitute/bio-nica/
I wrote my own implementation here.

Both algorithms are tested. 

Note that NICA only works when sources have a finite, non-zero probability to be exactly zero. So this would probably not work very well with gaussian or near-gaussian sources. 

In [ ]:
import numpy as np
from quadprog import solve_qp
import sys
import matplotlib.pyplot as plt
if "../" not in sys.path:
    sys.path.insert(0, "../")
    
from nica_algorithms import bio_nica_indirect, bio_nica_direct
    
import multiprocessing
from psutil import cpu_count
n_cpu = cpu_count(logical=False)

from utils.statistics import seed_from_gen

## Algorithm implementation
Embedded in an online simulation where random samples are successively presented. 

In [ ]:
# Learning rate as a function of time. 
def learn_decay(t, eta0, delta):
    return eta0 / (1.0 + delta * t)

In [ ]:
# Using the Github implementation of algorithm 1
def integrate_bionica1(m_init, l_init, update_bk, bk_init, bionica_params, bk_params, 
                     tmax, dt, seed=None, noisetype="uniform"):
    # Extract info about dimensionalities (notation K, D switched vs Lipshutz
    # to match the one used for IBCM).
    n_neu = m_init.shape[0]  # Number of ICA neurons N_K: number of extracted sources
    n_dim = m_init.shape[1]  # Number of input neurons N_D: number of mixtures
    n_int = l_init.shape[0]  # Number of interneurons N_I
    bk_vari_init, bk_vec_init = bk_init
    assert n_dim == bk_vec_init.shape[0], "Mismatch between dimension of m and background"
    learnrate0, decayrate, tau_avg = bionica_params
    
    # Initialize NICA algorithm object
    """ 
    s_dim         -- Dimension of sources
    x_dim         -- Dimension of mixtures
    n_dim         -- Dimension of interneurons
    P0            -- Initial guess for the lateral weight matrix P, must be of size s_dim by s_dim
    W0            -- Initial guess for the forward weight matrix W, must be of size s_dim by x_dim
    learning_rate -- Learning rate as a function of t
    tau           -- Learning rate factor for M (multiplier of the W learning rate)
    """
    nica_obj = bio_nica_indirect(n_neu, n_dim, n_int, eta0=learnrate0, decay=decayrate)

    rng = np.random.default_rng(seed=seed)
    tseries = np.arange(0, tmax, dt)

    # Containers for the solution over time
    m_series = np.zeros([tseries.shape[0], n_neu, n_dim])  # series of M^T (N_IxN_D)
    l_series = np.zeros([tseries.shape[0], n_int, n_neu])  # series of L (N_NxN_I)
    cbar_series = np.zeros([tseries.shape[0], n_neu])  # series of output projections
    bkvec_series = np.zeros([tseries.shape[0], n_dim])  # Input vecs, convenient to compute inhibited output
    bk_series = np.zeros([tseries.shape[0]] + list(bk_vari_init.shape))  # Sources (odor concentrations)
    
    ## Initialize running variables, separate from the containers above to avoid side effects.
    cbar = np.zeros(n_neu)  # inhibited neuron activities (after applying L, n)
    bk_vari = bk_vari_init.copy()
    bkvec = bk_vec_init.copy()
    mmat = m_init.copy()
    lmat = l_init.copy()

    # Store back some initial values in containers
    m_series[0] = m_init
    l_series[0] = l_init
    bkvec_series[0] = bkvec
    bk_series[0] = bk_vari

    # Generate required noise samples in advance
    if (tseries.shape[0]-1)*bk_vari.size > 1e7:
        raise ValueError("Too much memory needed; consider calling multiple times for shorter times")
    if noisetype == "normal":
        noises = rng.normal(0, 1, size=(tseries.shape[0]-1,*bk_vari.shape))
    elif noisetype == "uniform":
        noises = rng.random(size=(tseries.shape[0]-1, *bk_vari.shape))
    else:
        raise NotImplementedError("Noise option {} not implemented".format(noisetype))

    t = 0
    for k in range(0, len(tseries)-1):

        ## Update synaptic weights and background to time t+1 for next iteration
        cbar = nica_obj.fit_next(bkvec)
        cbar_series[k] = cbar
        
        # Update background to time k+1, to be used in next time step (k+1)
        bk_vari, bkvec = update_bk(bk_vari, bk_params, noises[k], dt)
        
        t += dt

        # Save synaptic weights at time step k+1
        knext = (k+1)
        m_series[knext] = nica_obj.W
        l_series[knext] = nica_obj.P.T
        bkvec_series[knext] = bkvec
        bk_series[knext] = bk_vari
    
    # Compute final neuronal activities with latest matrices and inputs
    cbar = nica_obj.fit_next(bkvec)
    cbar_series[-1] = cbar
    
    return tseries, bk_series, bkvec_series, m_series, l_series, cbar_series


In [ ]:
# Using the Github implementation of algorithm 1
def integrate_bionica2(m_init, l_init, update_bk, bk_init, bionica_params, bk_params, 
                     tmax, dt, seed=None, noisetype="uniform"):
    # Extract info about dimensionalities (notation K, D switched vs Lipshutz
    # to match the one used for IBCM).
    n_neu = m_init.shape[0]  # Number of ICA neurons N_K: number of extracted sources
    n_dim = m_init.shape[1]  # Number of input neurons N_D: number of mixtures
    n_int = l_init.shape[0]  # Number of interneurons N_I
    bk_vari_init, bk_vec_init = bk_init
    assert n_dim == bk_vec_init.shape[0], "Mismatch between dimension of m and background"
    learnrate0, decayrate, tau_avg, tau_l = bionica_params
    
    # Initialize NICA algorithm object
    """ 
    s_dim         -- Dimension of sources
    x_dim         -- Dimension of mixtures
    M0            -- Initial guess for the lateral weight matrix M, must be of size s_dim by s_dim
    W0            -- Initial guess for the forward weight matrix W, must be of size s_dim by x_dim
    learning_rate -- Learning rate as a function of t
    tau           -- Learning rate factor for M (multiplier of the W learning rate)
    """
    nica_obj = bio_nica_direct(n_neu, n_dim, eta0=learnrate0, decay=decayrate, tau=tau_l)

    rng = np.random.default_rng(seed=seed)
    tseries = np.arange(0, tmax, dt)

    # Containers for the solution over time
    m_series = np.zeros([tseries.shape[0], n_neu, n_dim])  # series of M^T (N_IxN_D)
    l_series = np.zeros([tseries.shape[0], n_int, n_neu])  # series of L (N_NxN_I)
    cbar_series = np.zeros([tseries.shape[0], n_neu])  # series of output projections
    bkvec_series = np.zeros([tseries.shape[0], n_dim])  # Input vecs, convenient to compute inhibited output
    bk_series = np.zeros([tseries.shape[0]] + list(bk_vari_init.shape))  # Sources (odor concentrations)
    
    ## Initialize running variables, separate from the containers above to avoid side effects.
    cbar = np.zeros(n_neu)  # inhibited neuron activities (after applying L, n)
    bk_vari = bk_vari_init.copy()
    bkvec = bk_vec_init.copy()
    mmat = m_init.copy()
    lmat = l_init.copy()

    # Store back some initial values in containers
    m_series[0] = m_init
    l_series[0] = l_init
    bkvec_series[0] = bkvec
    bk_series[0] = bk_vari

    # Generate required noise samples in advance
    if (tseries.shape[0]-1)*bk_vari.size > 1e7:
        raise ValueError("Too much memory needed; consider calling multiple times for shorter times")
    if noisetype == "normal":
        noises = rng.normal(0, 1, size=(tseries.shape[0]-1,*bk_vari.shape))
    elif noisetype == "uniform":
        noises = rng.random(size=(tseries.shape[0]-1, *bk_vari.shape))
    else:
        raise NotImplementedError("Noise option {} not implemented".format(noisetype))

    t = 0
    for k in range(0, len(tseries)-1):

        ## Update synaptic weights and background to time t+1 for next iteration
        cbar = nica_obj.fit_next(bkvec)
        cbar_series[k] = cbar
        
        # Update background to time k+1, to be used in next time step (k+1)
        bk_vari, bkvec = update_bk(bk_vari, bk_params, noises[k], dt)
        
        t += dt

        # Save synaptic weights at time step k+1
        knext = (k+1)
        m_series[knext] = nica_obj.W
        l_series[knext] = nica_obj.M
        bkvec_series[knext] = bkvec
        bk_series[knext] = bk_vari
    
    # Compute final neuronal activities with latest matrices and inputs
    cbar = nica_obj.fit_next(bkvec)
    cbar_series[-1] = cbar
    
    return tseries, bk_series, bkvec_series, m_series, l_series, cbar_series


### My own implementations, tabled for now, some problem. 

In [ ]:
# Algorithm 1 with interneurons. Assume P = L^T. 
# Modifications to Github's implementation:
#    - Using tau_avg averaging time rather than 1/current time: more realistic since no absolute zero time
#    - Working with L instead of P (P = L^T)
#    - Not checking for singular values of M, L every 100 steps
def integrate_bionica1(m_init, l_init, update_bk, bk_init, bionica_params, bk_params, 
                     tmax, dt, seed=None, noisetype="uniform"):
    # Extract info about dimensionalities (notation K, D switched vs Lipshutz
    # to match the one used for IBCM).
    n_neu = m_init.shape[0]  # Number of ICA neurons N_K: number of extracted sources
    n_dim = m_init.shape[1]  # Number of input neurons N_D: number of mixtures
    n_int = l_init.shape[0]  # Number of interneurons N_I
    bk_vari_init, bk_vec_init = bk_init
    assert n_dim == bk_vec_init.shape[0], "Mismatch between dimension of m and background"
    learnrate0, decayrate, tau_avg = bionica_params

    rng = np.random.default_rng(seed=seed)
    tseries = np.arange(0, tmax, dt)

    # Containers for the solution over time
    m_series = np.zeros([tseries.shape[0], n_neu, n_dim])  # series of M^T (N_IxN_D)
    l_series = np.zeros([tseries.shape[0], n_int, n_neu])  # series of L (N_NxN_I)
    cbar_series = np.zeros([tseries.shape[0], n_neu])  # series of output projections
    # No need to save interneuron activities, easily recovered as n = Ly
    bkvec_series = np.zeros([tseries.shape[0], n_dim])  # Input vecs, convenient to compute inhibited output
    bk_series = np.zeros([tseries.shape[0]] + list(bk_vari_init.shape))  # Sources (odor concentrations)
    
    ## Initialize running variables, separate from the containers above to avoid side effects.
    c = np.zeros(n_neu)  # un-inhibited neuron activities (before applying L)
    n = np.zeros(n_int)  # interneuron activities
    cbar = np.zeros(n_neu)  # inhibited neuron activities (after applying L, n)
    bk_vari = bk_vari_init.copy()
    bkvec = bk_vec_init.copy()
    mmat = m_init.copy()
    lmat = l_init.copy()
    
    # Running averages
    x_avg = bk_vec_init.copy()
    cbar_avg = cbar.copy()
    n_avg = n.copy()

    # Store back some initial values in containers
    m_series[0] = m_init
    l_series[0] = l_init
    bkvec_series[0] = bkvec
    bk_series[0] = bk_vari

    # Generate required noise samples in advance
    if (tseries.shape[0]-1)*bk_vari.size > 1e7:
        raise ValueError("Too much memory needed; consider calling multiple times for shorter times")
    if noisetype == "normal":
        noises = rng.normal(0, 1, size=(tseries.shape[0]-1,*bk_vari.shape))
    elif noisetype == "uniform":
        noises = rng.random(size=(tseries.shape[0]-1, *bk_vari.shape))
    else:
        raise NotImplementedError("Noise option {} not implemented".format(noisetype))
    
    # Constant matrices used often (don't re-create every iteration)
    identity_neu = np.eye(n_neu)
    zeros_neu = np.zeros(n_neu)
    newax = np.newaxis

    t = 0
    for k in range(0, len(tseries)-1):
        ## Compute neuronal activity at time t with background and matrices at t
        # Input projection with M
        c = mmat.dot(bkvec)
        
        # Fast iterations for cbar, n: quasi-static approximation, use quadprog
        # to optimize 1/2*cbar^T PL cbar - c^T cbar
        # subject to cbar \geq 0 at each element
        # Solution without constraint: cbar = (L^T L)^{-1} c
        cbar = solve_qp(G=lmat.T.dot(lmat), a=c, C=identity_neu, b=zeros_neu)[0]
        n = lmat.dot(cbar)
        
        # Save neuronal activities at time t (we don't save interneurons n)
        cbar_series[k] = cbar
        
        ## Update running averages of x, n, cbar up to time t
        x_avg += dt * (bkvec - x_avg) / tau_avg
        n_avg += dt * (n - n_avg) / tau_avg
        cbar_avg += dt * (cbar - cbar_avg) / tau_avg
        
        ## Update synaptic weights and background to time t+1 for next iteration
        # Based on neuronal activities, inputs, and running averages at time t
        # Learning rate at time t
        eta = learn_decay(t, learnrate0, decayrate)
        # Synaptic plasticity: update mmat, lmat to k+1 based on cbar, n at k
        mmat = mmat + dt*eta*((cbar - cbar_avg)[:, newax].dot((bkvec - x_avg)[newax, :]) - mmat)
        lmat = lmat + dt*eta*((n - n_avg)[:, newax].dot((cbar - cbar_avg)[newax, :]) - lmat)

        #if k % 100 == 0:
        #    for i in range(n_neu):
        #        if np.linalg.norm(mmat[i,:]) < .1:
        #            print(f'iteration {k}: M row {i} small: {mmat[i,:]}')
        #            mmat[i,:] = rng.standard_normal(n_dim)/np.sqrt(n_dim)

        # Update background to time k+1, to be used in next time step (k+1)
        bk_vari, bkvec = update_bk(bk_vari, bk_params, noises[k], dt)
        
        t += dt

        # Save synaptic weights at time step k+1
        knext = (k+1)
        m_series[knext] = mmat
        l_series[knext] = lmat
        bkvec_series[knext] = bkvec
        bk_series[knext] = bk_vari
    
    # Compute final neuronal activities with latest matrices and inputs
    c = mmat.dot(bkvec)
    cbar = solve_qp(G=lmat.T.dot(lmat), a=c, C=identity_neu, b=zeros_neu)[0]
    cbar_series[-1] = cbar
    
    return tseries, bk_series, bkvec_series, m_series, l_series, cbar_series


In [ ]:
# Algorithm 2 with two-compartment neurons
# Modifications to Github's implementation:
#    - Using tau_avg averaging time rather than 1/current time: more realistic since no absolute zero time
#    - Not checking for singular values of M every 100 steps
def integrate_bionica2(m_init, l_init, update_bk, bk_init, bionica_params, bk_params, 
                     tmax, dt, seed=None, noisetype="uniform"):
    # Extract info about dimensionalities (notation K, D switched vs Lipshutz
    # to match the one used for IBCM).
    n_neu = m_init.shape[0]  # Number of ICA neurons N_I: number of extracted sources
    n_dim = m_init.shape[1]  # Number of input neurons N_D: number of mixtures
    bk_vari_init, bk_vec_init = bk_init
    assert n_dim == bk_vec_init.shape[0], "Mismatch between dimension of m and background"
    learnrate0, decayrate, tau_avg, tau_l = bionica_params

    rng = np.random.default_rng(seed=seed)
    tseries = np.arange(0, tmax, dt)

    # Containers for the solution over time
    m_series = np.zeros([tseries.shape[0], n_neu, n_dim])  # series of M^T (N_IxN_D)
    l_series = np.zeros([tseries.shape[0], n_neu, n_neu])  # series of L (N_IxN_I)
    cbar_series = np.zeros([tseries.shape[0], n_neu])  # series of output projections
    bkvec_series = np.zeros([tseries.shape[0], n_dim])  # Input vecs, convenient to compute inhibited output
    bk_series = np.zeros([tseries.shape[0]] + list(bk_vari_init.shape))  # Sources (odor concentrations)
    
    ## Initialize running variables, separate from the containers above to avoid side effects.
    c = np.zeros(n_neu)  # un-inhibited neuron activities (before applying L)
    cbar = np.zeros(n_neu)  # inhibited neuron activities (after applying L, n)
    bk_vari = bk_vari_init.copy()
    bkvec = bk_vec_init.copy()
    mmat = m_init.copy()
    lmat = l_init.copy()
    
    # Running averages
    x_avg = bk_vec_init.copy()
    c_avg = c.copy()

    # Store back some initial values in containers
    m_series[0] = m_init
    l_series[0] = l_init
    bkvec_series[0] = bkvec
    bk_series[0] = bk_vari

    # Generate required noise samples in advance
    if (tseries.shape[0]-1)*bk_vari.size > 1e7:
        raise ValueError("Too much memory needed; consider calling multiple times for shorter times")
    if noisetype == "normal":
        noises = rng.normal(0, 1, size=(tseries.shape[0]-1,*bk_vari.shape))
    elif noisetype == "uniform":
        noises = rng.random(size=(tseries.shape[0]-1, *bk_vari.shape))
    else:
        raise NotImplementedError("Noise option {} not implemented".format(noisetype))
    
    # Constant matrices used often (don't re-create every iteration)
    identity_neu = np.eye(n_neu)
    zeros_neu = np.zeros(n_neu)
    newax = np.newaxis

    t = 0
    for k in range(0, len(tseries)-1):
        ## Compute neuronal activity at time t with background and matrices at t
        # Input projection with M
        c = mmat.dot(bkvec)
        
        # Fast iterations for cbar: quasi-static approximation, use quadprog
        # to optimize 1/2*cbar^T L cbar - c^T cbar
        # subject to cbar \geq 0 at each element
        # Solution without constraint: cbar = L^{+} c
        try:
            cbar = solve_qp(G=lmat, a=c, C=identity_neu, b=zeros_neu)[0]
        except:
            return tseries, bk_series, bkvec_series, m_series, l_series, cbar_series
        
        # Save neuronal activities at time t (we don't save interneurons n)
        cbar_series[k] = cbar
        
        ## Update running averages of x, n, cbar up to time t
        x_avg += dt * (bkvec - x_avg) / tau_avg
        c_avg += dt * (c - c_avg) / tau_avg
        
        ## Update synaptic weights and background to time t+1 for next iteration
        # Based on neuronal activities, inputs, and running averages at time t
        # Learning rate at t
        eta = learn_decay(t, learnrate0, decayrate)
        # Synaptic plasticity: update mmat, lmat to k+1 based on cbar, n at k
        mmat = mmat + 2.0*dt*eta*(cbar[:, newax].dot(bkvec[newax, :]) 
                                    - (c - c_avg)[:, newax].dot((bkvec - x_avg)[newax, :]))
        lmat = lmat + dt*eta/tau_l*(cbar[:, newax].dot(cbar[newax, :]) - lmat)
    
        # check to see if M is close to degenerate
        #if k%100==0:
        #    lam, v = np.linalg.eig(lmat)
        #    
        #    for i in range(n_neu):
        #        if lam[i]<1e-4:
        #            print(f'iteration {t}: close to degenerate')
        #            lam[i] = 1
        #        
        #    lmat = v@np.diag(lam)@v.T

        # Update background to time k+1, to be used in next time step (k+1)
        bk_vari, bkvec = update_bk(bk_vari, bk_params, noises[k], dt)
        
        t += dt

        # Save synaptic weights at time step k+1
        knext = (k+1)
        m_series[knext] = mmat
        l_series[knext] = lmat
        bkvec_series[knext] = bkvec
        bk_series[knext] = bk_vari
    
    # Compute final neuronal activities with latest matrices and inputs
    c = mmat.dot(bkvec)
    cbar = solve_qp(G=lmat, a=c, C=identity_neu, b=zeros_neu)[0]
    cbar_series[-1] = cbar
    
    return tseries, bk_series, bkvec_series, m_series, l_series, cbar_series


## Define the test conditions

### Sources
Each source is statistically independent. At every step, it is either set to zero with probability $\frac12$, or sampled uniformly from the interval $(0, \sqrt{48/5})$, with probability $\frac12$. 

Two synthetic test cases were considered: 3 sources and 10 sources. 


### Mixing matrix
Random square matrix with elements that are independent standard normal random variables.

They report a specific test instance in the appendix, but it's unclear whether this is actually what they used, because their Github code is not seeded and the mixing matrix isn't printed anywhere. So I will just generate my own random matrix anyways. 

### Initial values
Not specified in the paper, but based on the code on Github:
 - $M$ ($N_D \times N_K$) is a diagonal $D$ of ones stopping after $N_K \leq N_D$ columns, transformed by two random orthogonal matrices $R$ of appropriate dimensions: $M = R_{N_D} D R_{N_K}$. 
 - $L$ ($N_I \times N_K$) is a diagonal $D$ of ones stopping after $N_K \leq N_I$ columns


### Model parameters and rates
Specified in the appendix of the paper and hardcoded in algorithm class definitions on the Github repo. 
The learning rate is made time-dependent:

$$ \eta(t) = \frac{\eta_0}{1 + \delta t} $$

For the 3-source case and the 10-source case, algorithm 1:
 - Learning rate $\eta_0 = 0.01$
 - Decay rate: $\delta = 0.001$
 
For the 3-source case, algorithm 2:
 - Learning rate $\eta_0 = 0.1$
 - Decay rate: $\delta = 0.01$
 - $\tau$ for $L$ dynamics: $\tau = 0.8$

For the 10-source case, algorithm 2:
 - Learning rate $\eta_0 = 0.001$
 - Decay rate: $\delta = 10^{-4}$
 - $\tau$ for $L$ dynamics: $\tau = 0.03$
 
I will be using $\tau_{avg} = 150$, as in the IBCM model. 


### Error metric
Mean-squared error between the recovered sources, $\vec{\bar{c}}(t)$, and the original ones, $\vec{s}(t)$, with the best possible permutation of sources being performed. Assumes $N_I = N_K$, the number of sources is known and equal to the number of neurons/recovered sources. 

$$ E(t) = \frac{1}{t N_K} \sum_{t'=1}^{t} \|\vec{s}(t) - P \vec{\bar{c}}(t)  \|^2 $$

where $P$ is a permutation matrix optimized over all times $t$ (i.e. to find which source corresponds to which recovered source) to minimize the difference with $\vec{s}(t)$. 

The error is then computed as a function of time, as a moving exponential average, from the equation

$$ dE/dt = \frac{1}{1000}(E - err_t) $$

where $err_t$ is the error contribution of time point $t$. 

For this function, I simply copy-pasted the code available on Github from the bio-nica repository, adding a few comments for my own understanding. 


In [ ]:
# Code to generate a multivariate sample from N(0, 1) samples (stdnorm_vec)
def update_sources(bk_vari, bk_params, unif_vec, dt):
    # dt is an argument for compatibility, not used
    # smax should broadcast to the shape of bk_vari
    # zeroprob is assumed to be a float: the same max conc. for all sources
    mixmat, smax, zeroprob = bk_params
    # For each source, choose whether it is zero or non-zero
    # Reuse the uniform normal samples, conditioned on knowing they are in [zeroprob, 1.0)
    # to select the concentration of non-zero sources. 
    if zeroprob < 1.0:
        bk_vari = np.where(unif_vec < zeroprob, 0.0, (unif_vec - zeroprob) / (1.0 - zeroprob) * smax)
    else:  # This would be strange, but anyways. 
        bk_vari = np.zeros(mixmat.shape[1])
    return bk_vari, mixmat.dot(bk_vari)

# Generate a random orthogonal matrix
def random_orthogonal_mat(n, rng):
    # Tested in test_biopca notebook: this does give orthogonal matrices
    # Github code of bio-nica uses scipy.stats.ortho_group
    q, r = np.linalg.qr(rng.standard_normal(size=[n, n]), mode="complete")
    return q.dot(np.diagflat(np.sign(np.diagonal(r))))

def generate_mixing_mat(dim, rng):
    return rng.standard_normal(size=(dim, dim))

In [ ]:
# Found in https://github.com/flatironinstitute/bio-nica/blob/master/util.py
from scipy.optimize import linear_sum_assignment
def permutation_error(S_perm, Y):
    """
    Parameters:
    ====================
    S_perm   -- The data matrix of permuted sources
    Y   -- The data matrix of recovered sources
    
    Output:
    ====================
    err -- the (relative) Frobenius norm error
    """
    msg = ("The shape of the permuted sources S_perm"
            + "must equal the shape of the recovered sources Y")

    assert S_perm.shape==Y.shape, msg
    s_dim = S_perm.shape[0]
    iters = S_perm.shape[1]
    
    err = np.zeros(iters)
    
    # Determine the optimal permutation at the final time point.
    # We solve the linear assignment problem using the linear_sum_assignment package
    
    # Calculate cost matrix:
    C = np.zeros((s_dim, s_dim))
    
    # Compare every pair of recovered, original sources. 
    for i in range(s_dim):
        for j in range(s_dim):
            C[i,j] = ((S_perm[i] - Y[j])**2).sum()
    
    # Find the optimal assignment for the cost matrix C
    row_ind, col_ind = linear_sum_assignment(C)
    
    # Compute the error after each time point, moving exp. average over 1000 frames
    for t in range(iters):

        diff_t = (S_perm[row_ind[:],t] - Y[col_ind[:],t])**2
        error_t = diff_t.sum()/s_dim
        
        if t==0:
            err[t] = error_t
        elif t>0:
            # err[t] = err[t-1] + (error_t - err[t-1])/t
            err[t] = err[t-1] + (error_t - err[t-1])/1000
    
    return err

In [ ]:
def find_nearest_idx(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

## Reproduce tests cases
Both algorithms, both dimensionalities, with the prescribed parameters. 

Report average and ci across 10 trials with the same mixing matrix. 

In [ ]:
def run_onetest_twoalgos(init_m, init_l, update_fct, init_bk, nica_params1, nica_params2, back_params,  
                    duration, dt, seeds,  noisetype, tst):
    # Run simulation and compute error with algorithm 1
    # back_params: contains mixmat, smax, probzero
    res = integrate_bionica1(init_m, init_l, update_fct, init_bk, nica_params1, back_params,  
                duration, dt, seed=seeds[0], noisetype=noisetype)
    tser, bkser, bkvecser, mser, lser, cbarser = res
    err_series1 = permutation_error(cbarser.T, bkser.T)  # Each column is a time point
    
    # Run simulation and compute error with algorithm 2
    res = integrate_bionica2(init_m, init_l, update_fct, init_bk, nica_params2, back_params,  
                duration, dt, seed=seeds[1], noisetype=noisetype)
    tser, bkser, bkvecser, mser, lser, cbarser = res
    
    err_series2 = permutation_error(cbarser.T, bkser.T)
    
    print("Plotting")
    fig, axes = plt.subplots(3)
    n_pts = tser.size
    axes[0].plot(bkser[n_pts//2:, 1], bkser[n_pts//2:, 2], ls="none", marker="o", ms=2)
    axes[1].plot(bkvecser[n_pts//2:, 1], bkvecser[n_pts//2:, 2], ls="none", marker="o", ms=2)
    axes[2].plot(cbarser[n_pts//2:, 1], cbarser[n_pts//2:, 2], ls="none", marker="o", ms=2)
    plt.show()
    plt.close()
    print("Finished plot")
    
    # Check distribution of samples
    #msg = "Samples are not zero with probability {}".format(back_params[-1])
    #assert abs(np.count_nonzero(bkser)/bkser.size/(1-back_params[-1]) - 1.0) < 2.0 / np.sqrt(tser.size), msg
    
    print("Finished test ntst = {}".format(tst))
    
    return tser, err_series1, err_series2

In [ ]:
## Parallelized version of the tests
# Run ntst tests of both algorithms (1 and 2) for a choice of N_D (n mix), N_K (n sources)
# Keep the duration of each test to 1e4 by default, to make things quicker. 
def run_tests_nkchoice_parallel(ntst, nica_params1, nica_params2, mix_params, 
            duration=1e4, rng=None, njob=n_cpu):
    # Extract a few parameters
    #mix_mat, s_max, zero_prob = mix_params
    #learnrate0, decayrate, tau_avg, tau_l = nica_params
    nd = mix_params[0].shape[0]  # n_dim = N_D = number of mixtures
    nk = mix_params[0].shape[1]  # n_sources = N_K = number of ICA neurons
    nn = nk  # Use a number of interneurons equal to the number of sources n_k
    
    if rng is None:
        rng = np.random.default_rng()
    
    # Initial matrices. M: diagonal of ones stopping after N_K columns
    # and transformed by two random orthogonal matrices
    init_m = (random_orthogonal_mat(nk, rng)
              .dot(np.eye(nk, nd))
              .dot(random_orthogonal_mat(nd, rng)))
    # L: identity matrix
    init_l1 = np.eye(nn, nk)
    dt = 1.0
    
    # To generate a new seed for each simulation
    seed_sequence = np.random.SeedSequence(seed_from_gen(rng))
    
    # Pool of workers, to parallelize
    pool = multiprocessing.Pool(njob)
    res_objs = []
    
    for tst in range(ntst):
        
        # Initialize background: use update_sources
        init_bk = list(update_sources(None, mix_params, rng.random(size=nk), dt))
        
        # Get new seeds
        seeds_tst = seed_sequence.spawn(2)
        
        # Launch this test in parallel of others
        res = pool.apply_async(run_onetest_twoalgos, 
                            args=(init_m, init_l1, update_sources, init_bk, nica_params1, nica_params2,
                            mix_params, duration, dt, seeds_tst, "uniform", tst)
                            )
        res_objs.append(res)
    
    # Get the results
    res_objs_finished = [a.get() for a in res_objs]
    
    # Don't forget to close the Pool! 
    pool.close()
    
    # Store results in appropriate containers
    chosen_t = res_objs_finished[0][0]
    err_algo1 = np.zeros([ntst, len(chosen_t)])
    err_algo2 = np.zeros([ntst, len(chosen_t)])
    for tst in range(ntst):
        err_algo1[tst] = res_objs_finished[tst][1]
        err_algo2[tst] = res_objs_finished[tst][2]
        
    return chosen_t, err_algo1, err_algo2

## Test runs for small $N_D$, $N_K$
$N_K = 3$, $N_D = 4$. 

Reminder of parameters:

For the 3-source case and the 10-source case, algorithm 1:
 - Learning rate $\eta_0 = 0.01$
 - Decay rate: $\delta = 0.001$
 
For the 3-source case, algorithm 2:
 - Learning rate $\eta_0 = 0.1$
 - Decay rate: $\delta = 0.01$
 - $\tau$ for $L$ dynamics: $\tau = 0.8$

In [ ]:
rgen = np.random.default_rng(seed=0x508f6659e430c228a2bdeababc96b8de)

# Dimensionalities
n_n = 3  # input: number of mixtures
n_k = 3  # output: number of sources

mix_params_smallk = [
    rgen.standard_normal(size=[n_n, n_k]),   # mix_mat
    np.sqrt(48/5),                           # s_max
    0.5                                      # zero_prob
]

#learnrate0, decayrate, tau_avg, tau_l = nica_params
bionica_params1_smallk = [0.01, 0.001, 150.0]
bionica_params2_smallk = [0.1, 0.01, 150.0, 0.8]

In [ ]:
# Run tests
n_tests = 1
duration = 1e5

try:
    res_file = np.load("outputs/test_bionica_k3_permutation_errors.npz")
except FileNotFoundError:
    print("Launching test...")
    #ntst, nica_params1, nica_params2, mix_params, 
            #duration=1e4, rng=None, njob=n_cpu
    chosen_times, errors_algo1, errors_algo2 = run_tests_nkchoice_parallel(
                    n_tests, bionica_params1_smallk, bionica_params2_smallk, 
                    mix_params_smallk, duration=duration, rng=rgen, njob=n_cpu)
    # Write test results to disk so we don't have to run code every time
    #np.savez("outputs/test_bionica_k3_permutation_errors.npz", 
    #     chosen_times=chosen_times, 
    #     errors_algo1=errors_algo1, 
    #     errors_algo2=errors_algo2
    #    )
    print("New test results saved.")
else:
    print("Managed to load existing test results.")
    chosen_times = res_file["chosen_times"]
    errors_algo1 = res_file["errors_algo1"]
    errors_algo2 = res_file["errors_algo2"]
    res_file.close()


In [ ]:
# Plot the median alignment error as a function of iteration number
mederr_algo1 = np.median(errors_algo1, axis=0)
mederr_algo2 = np.median(errors_algo2, axis=0)
fig, ax = plt.subplots()
start_t = find_nearest_idx(chosen_times, 1e2)
li1, = ax.plot(chosen_times[start_t:], mederr_algo1[start_t:], label="Bio-NICA 1")
li2, = ax.plot(chosen_times[start_t:], mederr_algo2[start_t:], label="Bio-NICA 2", ls="--")
ax.set(xlabel="Iterations", ylabel="Subspace alignment error", 
       yscale="log", xscale="log")

ax.legend()
fig.tight_layout()
#fig.savefig("../figures/tests/test_ifpsp_n10_k3_align_subspace_error.pdf", 
#           transparent="True", bbox_inches="tight")
plt.show()
plt.close()

## Test runs for large N, K

In [ ]:
# Dimensionalities
n_n2 = 10  # input: mixtures
n_k2 = 10  # output: sources

mix_params_largek = [
    rgen.standard_normal(size=[n_n2, n_k2]),   # mix_mat
    np.sqrt(48/5),                           # s_max
    0.5                                      # zero_prob
]

#learnrate0, decayrate, tau_avg, tau_l = nica_params
bionica_params1_largek = [0.01, 0.001, 150.0]
bionica_params2_largek = [0.001, 0.0001, 150.0, 0.03]

In [ ]:
# Run tests
n_tests = 1
duration = 1e5

try:
    res_file = np.load("outputs/test_bionica_k10_permutation_errors.npz")
except FileNotFoundError:
    print("Launching test...")
    #ntst, nica_params1, nica_params2, mix_params, 
            #duration=1e4, rng=None, njob=n_cpu
    chosen_times2, errors_algo1_large, errors_algo2_large = run_tests_nkchoice_parallel(
                    n_tests, bionica_params1_largek, bionica_params2_largek, 
                    mix_params_largek, duration=duration, rng=rgen, njob=n_cpu)
    # Write test results to disk so we don't have to run code every time
    #np.savez("outputs/test_bionica_k10_permutation_errors.npz", 
    #     chosen_times=chosen_times, 
    #     errors_algo1=errors_algo1, 
    #     errors_algo2=errors_algo2
    #    )
    print("New test results saved.")
else:
    print("Managed to load existing test results.")
    chosen_times2 = res_file["chosen_times"]
    errors_algo1_large = res_file["errors_algo1"]
    errors_algo2_large = res_file["errors_algo2"]
    res_file.close()


In [ ]:
# Plot the median alignment error as a function of iteration number
mederr_algo1_large = np.median(errors_algo1_large, axis=0)
mederr_algo2_large = np.median(errors_algo2_large, axis=0)
fig, ax = plt.subplots()
li1, = ax.plot(chosen_times2, mederr_algo1_large, label="BioNICA 1")
li2, = ax.plot(chosen_times2, mederr_algo2_large, label="BioNICA 2", ls="--")
ax.set(xlabel="Iterations", ylabel="Subspace alignment error", 
       yscale="log", xscale="log")

ax.legend()
fig.tight_layout()
#fig.savefig("../figures/tests/test_bionica_n100_k10_align_subspace_error.pdf", 
#           transparent="True", bbox_inches="tight")
plt.show()
plt.close()